# Import the packages and check connection to bucket

In [1]:
from google.cloud import storage
import pandas as pd ## for dataset and eda
import numpy as np ## for eda
from datetime import datetime
from tqdm import tqdm

In [2]:
bucket_name = "firstprojectdl"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

print(bucket)
print('Great, we now have access to our first bucket on google cloud storage where we put our data')

<Bucket: firstprojectdl>
Great, we now have access to our first bucket on google cloud storage where we put our data


In [3]:
from google.cloud import storage
import pandas as pd

bucket_name = "firstprojectdl"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "data/movieLens/movieLens20M/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')

dfDict = {}
dateparse = lambda x: datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')

for blob in blobs:
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        print(file_name)
        if file_name =='ratings20M.dat':
            df = pd.read_csv(file_name, sep=',', 
                            names=['user_id', 'movie_id', 'rating', 'timestamp'],
                            skiprows = 1, 
                            parse_dates=['timestamp'], 
                            date_parser=dateparse) # load the rating data
 

ratings20M.dat


## Memory Error with 20 million dataset then we will only read the file with first 2.5 million rows

In [4]:
df.head()

,user_id,movie_id,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [5]:
df = df.loc[(df['movie_id'].isin(range(15000))) & (df['user_id'].isin(range(15000))), :]

In [6]:
df.head()

,user_id,movie_id,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [7]:
listMovies = np.sort(df.movie_id.unique())
ind = 0
for movies in tqdm(listMovies):
    df.loc[df['movie_id'] == movies, 'movie_id'] = ind
    ind += 1

100%|██████████| 8190/8190 [01:12<00:00, 112.28it/s]


In [8]:
listMovies = df.movie_id.unique()
len(listMovies)


8190

In [9]:
listMovies = df.movie_id.unique()
len(listMovies)

8190

In [65]:
listMovies.sort()

In [10]:
df.to_csv('ratings2.5M.dat')

In [11]:
df.head()

,user_id,movie_id,rating,timestamp
0,1,1,3.5,2005-04-02 23:53:47
1,1,28,3.5,2005-04-02 23:31:16
2,1,31,3.5,2005-04-02 23:33:39
3,1,46,3.5,2005-04-02 23:32:07
4,1,49,3.5,2005-04-02 23:29:40


(1960385, 4)

In [12]:
num_users = df.user_id.unique().shape[0]
print(f'The datasets presents {num_users} users')

The datasets presents 14998 users


In [13]:
num_ratings = df.rating.shape[0]
print(f'The datasets presents {num_ratings} distinct ratings')

The datasets presents 1960385 distinct ratings


In [14]:
num_items = df.movie_id.unique().shape[0]
print(f'The datasets presents {num_items} items')

The datasets presents 8190 items


In [15]:
df.movie_id.unique()

array([   1,   28,   31, ..., 7205, 7697, 5945])

In [17]:
with open('ratings2.5M.dat') as infile:
  with open('ratings2.5MNew.dat', 'w') as outfile:
    for line in infile:
      fields = line.split(',')
      outfile.write('::'.join(fields))

In [19]:
fp = open("ratings2.5MNew.dat")
lines = fp.readlines()

In [20]:
lines[0]

'::user_id::movie_id::rating::timestamp\n'

In [1]:
import numpy as np

In [2]:
# Comet ML for experiment logging
from comet_ml import Experiment

In [3]:
# Import packages
import tensorflow as tf

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [4]:
# Set random seed
tf.random.set_random_seed(1994)
np.random.seed(1994)

In [5]:
def read_rating(num_users, num_items, num_total_ratings, a, b, train_ratio):
    """
    Function to read in the ratings data
    :param path: path to ratings data
    :param num_users: number of users
    :param num_items: number of items
    :param num_total_ratings: number of total ratings
    :param a: positive rating (1)
    :param b: negative rating (0)
    :param train_ratio: ratio that splits train and test sets
    """
    fp = open("ratings2.5MNew.dat")

    user_train_set = set()
    user_test_set = set()
    item_train_set = set()
    item_test_set = set()
    R = np.zeros((num_users, num_items))
    mask_R = np.zeros((num_users, num_items))
    C = np.ones((num_users, num_items)) * b

    train_R = np.zeros((num_users, num_items))
    test_R = np.zeros((num_users, num_items))

    train_mask_R = np.zeros((num_users, num_items))
    test_mask_R = np.zeros((num_users, num_items))

    random_perm_idx = np.random.permutation(num_total_ratings)
    train_idx = random_perm_idx[0:int(num_total_ratings * train_ratio)]
    test_idx = random_perm_idx[int(num_total_ratings * train_ratio):]

    num_train_ratings = len(train_idx)
    num_test_ratings = len(test_idx)

    lines = fp.readlines()
    for line in lines[1:]:
        _, user, item, rating, _ = line.split("::")
        user_idx = int(user) - 1
        item_idx = int(item) - 1
        R[user_idx, item_idx] = float(rating)
        mask_R[user_idx, item_idx] = 1
        C[user_idx, item_idx] = a

    # Training set
    for itr in train_idx:
        line = lines[itr]
        _, user, item, rating, _ = line.split("::")
        if (user == 'user_id') | (item == 'movie_id') | (rating == 'rating'):
            continue
        user_idx = int(user) - 1
        item_idx = int(item) - 1
        train_R[user_idx, item_idx] = float(rating)
        train_mask_R[user_idx, item_idx] = 1

        user_train_set.add(user_idx)
        item_train_set.add(item_idx)

    # Test set
    for itr in test_idx:
        line = lines[itr]
        _, user, item, rating, _ = line.split("::")
        if (user == 'user_id') | (item == 'movie_id') | (rating == 'rating'):
            continue
        user_idx = int(user) - 1
        item_idx = int(item) - 1
        test_R[user_idx, item_idx] = float(rating)
        test_mask_R[user_idx, item_idx] = 1

        user_test_set.add(user_idx)
        item_test_set.add(item_idx)

    return R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R, num_train_ratings, num_test_ratings, \
           user_train_set, item_train_set, user_test_set, item_test_set

In [6]:
data_name = 'ml-2.5M'
num_users = 14999
num_items = 8190
num_total_ratings = 1960385
# Data is split into random 75% - 25% train-test sets
train_ratio = 0.75

In [7]:
R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R, num_train_ratings, num_test_ratings, \
user_train_set, item_train_set, user_test_set, item_test_set = read_rating(num_users, num_items, num_total_ratings, 1, 0, train_ratio)

In [10]:
# Get hyper-parameters
hyper_params = {
    "hidden_neuron": 500,
    "lambda_value": 1,
    "epochs": 500,
    "batch_size": 512,
    "optimizer": "Adam",
    "learning_rate": 0.001,
    "random_seed": 1994
}

In [13]:
# Import packages
import tensorflow as tf
import time
import numpy as np
import os
import math

In [14]:
class AutoRec:
    """
    Function to define the AutoRec model class
    """
    def __init__(self, sess, args,
                 num_users, num_items,
                 R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R, num_train_ratings, num_test_ratings,
                 user_train_set, item_train_set, user_test_set, item_test_set,
                 result_path):

        self.sess = sess
        self.args = args

        self.num_users = num_users
        self.num_items = num_items

        self.R = R
        self.mask_R = mask_R
        self.C = C
        self.train_R = train_R
        self.train_mask_R = train_mask_R
        self.test_R = test_R
        self.test_mask_R = test_mask_R
        self.num_train_ratings = num_train_ratings
        self.num_test_ratings = num_test_ratings

        self.user_train_set = user_train_set
        self.item_train_set = item_train_set
        self.user_test_set = user_test_set
        self.item_test_set = item_test_set

        self.hidden_neuron = args['hidden_neuron']
        self.train_epoch = args['train_epoch']
        self.batch_size = args['batch_size']
        self.num_batch = int(math.ceil(self.num_users / float(self.batch_size)))

        self.base_lr = args['base_lr']
        self.optimizer_method = args['optimizer_method']
        self.display_step = args['display_step']
        self.random_seed = args['random_seed']

        self.global_step = tf.Variable(0, trainable=False)
        self.decay_epoch_step = args['decay_epoch_step']
        self.decay_step = self.decay_epoch_step * self.num_batch
        self.lr = tf.compat.v1.train.exponential_decay(self.base_lr, self.global_step,
                                                       self.decay_step, 0.96, staircase=True)
        self.lambda_value = args['lambda_value']

        self.train_cost_list = []
        self.test_cost_list = []
        self.test_rmse_list = []

        self.result_path = result_path
        self.grad_clip = args['grad_clip']

    def run(self, experiment):
        """
        Function to run AutoRec
        :param experiment: CometML Experiment function
        """
        # Build AutoRec
        self.prepare_model()
        init = tf.compat.v1.global_variables_initializer()
        self.sess.run(init)

        # Train and evaluate AutoRec for all epochs
        for epoch_itr in range(self.train_epoch):
            experiment.set_step(epoch_itr)
            self.train_model(epoch_itr,experiment)
            self.test_model(epoch_itr, experiment)

        # Log results
        self.make_records()

    def prepare_model(self):
        """
        Function to build AutoRec
        """
        self.input_R = tf.compat.v1.placeholder(dtype=tf.float32,
                                                shape=[None, self.num_items],
                                                name="input_R")
        self.input_mask_R = tf.compat.v1.placeholder(dtype=tf.float32,
                                                     shape=[None, self.num_items],
                                                     name="input_mask_R")

        V = tf.compat.v1.get_variable(name="V", initializer=tf.compat.v1.truncated_normal(
            shape=[self.num_items, self.hidden_neuron],
            mean=0, stddev=0.03), dtype=tf.float32)
        W = tf.compat.v1.get_variable(name="W", initializer=tf.compat.v1.truncated_normal(
            shape=[self.hidden_neuron, self.num_items],
            mean=0, stddev=0.03), dtype=tf.float32)
        mu = tf.compat.v1.get_variable(name="mu", initializer=tf.zeros(shape=self.hidden_neuron), dtype=tf.float32)
        b = tf.compat.v1.get_variable(name="b", initializer=tf.zeros(shape=self.num_items), dtype=tf.float32)

        pre_Encoder = tf.matmul(self.input_R, V) + mu
        self.Encoder = tf.nn.sigmoid(pre_Encoder)
        pre_Decoder = tf.matmul(self.Encoder, W) + b
        self.Decoder = tf.identity(pre_Decoder)

        pre_rec_cost = tf.multiply((self.input_R - self.Decoder), self.input_mask_R)
        rec_cost = tf.square(self.l2_norm(pre_rec_cost))
        pre_reg_cost = tf.square(self.l2_norm(W)) + tf.square(self.l2_norm(V))
        reg_cost = self.lambda_value * 0.5 * pre_reg_cost

        self.cost = rec_cost + reg_cost

        if self.optimizer_method == "Adam":
            optimizer = tf.compat.v1.train.AdamOptimizer(self.lr)
        elif self.optimizer_method == "RMSProp":
            optimizer = tf.compat.v1.train.RMSPropOptimizer(self.lr)
        else:
            raise ValueError("Optimizer Key ERROR")

        if self.grad_clip:
            gvs = optimizer.compute_gradients(self.cost)
            capped_gvs = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gvs]
            self.optimizer = optimizer.apply_gradients(capped_gvs, global_step=self.global_step)
        else:
            self.optimizer = optimizer.minimize(self.cost, global_step=self.global_step)

    def train_model(self, itr, experiment):
        """
        Function to train AutoRec
        :param itr: Current iteration
        :param experiment: CometML experiment
        """
        start_time = time.time()
        random_perm_doc_idx = np.random.permutation(self.num_users)

        batch_cost = 0
        for i in range(self.num_batch):
            if i == self.num_batch - 1:
                batch_set_idx = random_perm_doc_idx[i * self.batch_size:]
            elif i < self.num_batch - 1:
                batch_set_idx = random_perm_doc_idx[i * self.batch_size: (i + 1) * self.batch_size]

            _, Cost = self.sess.run(
                [self.optimizer, self.cost],
                feed_dict={self.input_R: self.train_R[batch_set_idx, :],
                           self.input_mask_R: self.train_mask_R[batch_set_idx, :]})

            batch_cost = batch_cost + Cost
        self.train_cost_list.append(batch_cost)

        if (itr + 1) % self.display_step == 0:

            print("Training //", "Epoch %d //" % (itr), " Total cost = {:.2f}".format(batch_cost),
                  "Elapsed time : %d sec" % (time.time() - start_time))

        experiment.log_metric("loss", batch_cost, step=itr)

    def test_model(self, itr, experiment):
        """
        Function to evaluate AutoRec
        :param itr: Current iteration
        :param experiment: CometML experiment
        """
        start_time = time.time()
        Cost, Decoder = self.sess.run(
            [self.cost, self.Decoder],
            feed_dict={self.input_R: self.test_R,
                       self.input_mask_R: self.test_mask_R})

        self.test_cost_list.append(Cost)

        if (itr + 1) % self.display_step == 0:
            Estimated_R = Decoder.clip(min=1, max=5)
            unseen_user_test_list = list(self.user_test_set - self.user_train_set)
            unseen_item_test_list = list(self.item_test_set - self.item_train_set)

            for user in unseen_user_test_list:
                for item in unseen_item_test_list:
                    if self.test_mask_R[user, item] == 1:  # exist in test set
                        Estimated_R[user, item] = 3

            pre_numerator = np.multiply((Estimated_R - self.test_R), self.test_mask_R)
            numerator = np.sum(np.square(pre_numerator))
            denominator = self.num_test_ratings
            RMSE = np.sqrt(numerator / float(denominator))

            self.test_rmse_list.append(RMSE)

            print("Testing //", "Epoch %d //" % (itr), " Total cost = {:.2f}".format(Cost),
                  " RMSE = {:.5f}".format(RMSE),
                  "Elapsed time : %d sec" % (time.time() - start_time))
            print("=" * 100)

        experiment.log_metric("RMSE", RMSE, step=itr)

    def make_records(self):
        """
        Function to log results
        """
        if not os.path.exists(self.result_path):
            os.makedirs(self.result_path)

        basic_info = self.result_path + "basic_info.txt"
        train_record = self.result_path + "train_record.txt"
        test_record = self.result_path + "test_record.txt"

        with open(train_record, 'w') as f:
            f.write(str("Cost:"))
            f.write('\t')
            for itr in range(len(self.train_cost_list)):
                f.write(str(self.train_cost_list[itr]))
                f.write('\t')
            f.write('\n')

        with open(test_record, 'w') as g:
            g.write(str("Cost:"))
            g.write('\t')
            for itr in range(len(self.test_cost_list)):
                g.write(str(self.test_cost_list[itr]))
                g.write('\t')
            g.write('\n')

            g.write(str("RMSE:"))
            for itr in range(len(self.test_rmse_list)):
                g.write(str(self.test_rmse_list[itr]))
                g.write('\t')
            g.write('\n')

        #with open(basic_info, 'w') as h:
        #    h.write(str(self.args))

    def l2_norm(self, tensor):
        """
        Function to apply L2 normalization
        :param tensor: TensorFlow tensor
        """
        return tf.sqrt(tf.reduce_sum(tf.square(tensor)))

In [15]:
# Initialize TensorFlow Config
config = tf.compat.v1.ConfigProto()

In [16]:
experiment = Experiment(api_key="vV3GGUE8tetqNJuD2kGxVi1M9", project_name="autoencoders-movielens2.5M")
experiment.log_parameters(hyper_params)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/benichou/autoencoders-movielens100k/335bfcaa053248e7b8e970b24682bf82



In [17]:
args = {'hidden_neuron': 500, 'lambda_value':1, 'train_epoch': 500, 'batch_size': 512, 'optimizer_method': 'Adam', 'grad_clip': False, 'base_lr': 1e-3, 'decay_epoch_step': 50, 'random_seed':1994, 'display_step':1}

In [18]:
result_path = 'results/'

In [ ]:
with tf.compat.v1.Session(config=config) as sess:
    # Define the AutoRec class from `AutoRec.py`
    AutoRec = AutoRec(sess, args,
                      num_users, num_items,
                      R, mask_R, C, train_R, train_mask_R, test_R, test_mask_R, num_train_ratings, num_test_ratings,
                      user_train_set, item_train_set, user_test_set, item_test_set,
                      result_path)
    # Run the AutoRec model
    AutoRec.run(experiment)

COMET ERROR: Failed to extract parameters from Optimizer.init()


Training // Epoch 0 //  Total cost = 4719396.64 Elapsed time : 7 sec
Testing // Epoch 0 //  Total cost = 765908.25  RMSE = 1.24734 Elapsed time : 4 sec
Training // Epoch 1 //  Total cost = 1564239.07 Elapsed time : 6 sec
Testing // Epoch 1 //  Total cost = 581948.88  RMSE = 1.08677 Elapsed time : 4 sec
Training // Epoch 2 //  Total cost = 1438181.81 Elapsed time : 6 sec
Testing // Epoch 2 //  Total cost = 531821.19  RMSE = 1.03873 Elapsed time : 4 sec
Training // Epoch 3 //  Total cost = 1399633.07 Elapsed time : 6 sec
Testing // Epoch 3 //  Total cost = 509665.81  RMSE = 1.01687 Elapsed time : 4 sec
Training // Epoch 4 //  Total cost = 1365781.86 Elapsed time : 6 sec
Testing // Epoch 4 //  Total cost = 497312.34  RMSE = 1.00453 Elapsed time : 4 sec
Training // Epoch 5 //  Total cost = 1333254.08 Elapsed time : 6 sec
Testing // Epoch 5 //  Total cost = 488337.50  RMSE = 0.99540 Elapsed time : 4 sec
Training // Epoch 6 //  Total cost = 1304383.24 Elapsed time : 6 sec
Testing // Epoch 6 

Testing // Epoch 32 //  Total cost = 412633.00  RMSE = 0.91351 Elapsed time : 4 sec
Training // Epoch 33 //  Total cost = 697150.00 Elapsed time : 6 sec
Testing // Epoch 33 //  Total cost = 409468.19  RMSE = 0.90984 Elapsed time : 4 sec
Training // Epoch 34 //  Total cost = 677632.35 Elapsed time : 6 sec
Testing // Epoch 34 //  Total cost = 406654.78  RMSE = 0.90660 Elapsed time : 4 sec
Training // Epoch 35 //  Total cost = 658426.97 Elapsed time : 7 sec
Testing // Epoch 35 //  Total cost = 404691.97  RMSE = 0.90428 Elapsed time : 4 sec
Training // Epoch 36 //  Total cost = 640898.99 Elapsed time : 6 sec
Testing // Epoch 36 //  Total cost = 403427.59  RMSE = 0.90273 Elapsed time : 4 sec
Training // Epoch 37 //  Total cost = 624536.85 Elapsed time : 6 sec
Testing // Epoch 37 //  Total cost = 401204.41  RMSE = 0.90015 Elapsed time : 4 sec
Training // Epoch 38 //  Total cost = 607322.76 Elapsed time : 6 sec
Testing // Epoch 38 //  Total cost = 397431.78  RMSE = 0.89574 Elapsed time : 4 se

Training // Epoch 65 //  Total cost = 354186.69 Elapsed time : 8 sec
Testing // Epoch 65 //  Total cost = 363354.50  RMSE = 0.85426 Elapsed time : 4 sec
Training // Epoch 66 //  Total cost = 349743.47 Elapsed time : 8 sec
Testing // Epoch 66 //  Total cost = 361275.53  RMSE = 0.85163 Elapsed time : 4 sec
Training // Epoch 67 //  Total cost = 345518.00 Elapsed time : 8 sec
Testing // Epoch 67 //  Total cost = 361914.25  RMSE = 0.85248 Elapsed time : 4 sec
Training // Epoch 68 //  Total cost = 341219.81 Elapsed time : 8 sec
Testing // Epoch 68 //  Total cost = 359629.03  RMSE = 0.84957 Elapsed time : 4 sec
Training // Epoch 69 //  Total cost = 338059.11 Elapsed time : 8 sec
Testing // Epoch 69 //  Total cost = 360310.78  RMSE = 0.85043 Elapsed time : 4 sec
Training // Epoch 70 //  Total cost = 334459.25 Elapsed time : 7 sec
Testing // Epoch 70 //  Total cost = 358534.44  RMSE = 0.84816 Elapsed time : 4 sec
Training // Epoch 71 //  Total cost = 330709.62 Elapsed time : 6 sec
Testing // Ep

Testing // Epoch 97 //  Total cost = 343913.59  RMSE = 0.82966 Elapsed time : 4 sec
Training // Epoch 98 //  Total cost = 267194.94 Elapsed time : 7 sec
Testing // Epoch 98 //  Total cost = 344507.69  RMSE = 0.83043 Elapsed time : 4 sec
Training // Epoch 99 //  Total cost = 266226.47 Elapsed time : 7 sec
Testing // Epoch 99 //  Total cost = 342221.53  RMSE = 0.82739 Elapsed time : 4 sec
Training // Epoch 100 //  Total cost = 264423.47 Elapsed time : 7 sec
Testing // Epoch 100 //  Total cost = 341436.56  RMSE = 0.82646 Elapsed time : 4 sec
Training // Epoch 101 //  Total cost = 262531.26 Elapsed time : 8 sec
Testing // Epoch 101 //  Total cost = 339984.19  RMSE = 0.82466 Elapsed time : 4 sec
Training // Epoch 102 //  Total cost = 260986.78 Elapsed time : 8 sec
Testing // Epoch 102 //  Total cost = 341035.94  RMSE = 0.82596 Elapsed time : 4 sec
Training // Epoch 103 //  Total cost = 259241.03 Elapsed time : 8 sec
Testing // Epoch 103 //  Total cost = 340158.31  RMSE = 0.82482 Elapsed tim

Training // Epoch 130 //  Total cost = 239377.46 Elapsed time : 8 sec
Testing // Epoch 130 //  Total cost = 330957.97  RMSE = 0.81272 Elapsed time : 4 sec
Training // Epoch 131 //  Total cost = 238585.68 Elapsed time : 8 sec
Testing // Epoch 131 //  Total cost = 332016.84  RMSE = 0.81444 Elapsed time : 4 sec
Training // Epoch 132 //  Total cost = 237394.35 Elapsed time : 8 sec
Testing // Epoch 132 //  Total cost = 333000.25  RMSE = 0.81558 Elapsed time : 4 sec
Training // Epoch 133 //  Total cost = 236646.07 Elapsed time : 8 sec
Testing // Epoch 133 //  Total cost = 331416.31  RMSE = 0.81360 Elapsed time : 4 sec
Training // Epoch 134 //  Total cost = 235818.22 Elapsed time : 8 sec
Testing // Epoch 134 //  Total cost = 330988.50  RMSE = 0.81304 Elapsed time : 4 sec
Training // Epoch 135 //  Total cost = 235700.80 Elapsed time : 8 sec
Testing // Epoch 135 //  Total cost = 333239.91  RMSE = 0.81609 Elapsed time : 4 sec
Training // Epoch 136 //  Total cost = 236682.08 Elapsed time : 8 sec


Testing // Epoch 162 //  Total cost = 325595.56  RMSE = 0.80612 Elapsed time : 4 sec
Training // Epoch 163 //  Total cost = 224023.94 Elapsed time : 8 sec
Testing // Epoch 163 //  Total cost = 326417.97  RMSE = 0.80735 Elapsed time : 4 sec
Training // Epoch 164 //  Total cost = 223277.02 Elapsed time : 8 sec
Testing // Epoch 164 //  Total cost = 325131.00  RMSE = 0.80545 Elapsed time : 4 sec
Training // Epoch 165 //  Total cost = 223506.50 Elapsed time : 8 sec
Testing // Epoch 165 //  Total cost = 325515.53  RMSE = 0.80612 Elapsed time : 4 sec
Training // Epoch 166 //  Total cost = 223353.61 Elapsed time : 8 sec
Testing // Epoch 166 //  Total cost = 325215.50  RMSE = 0.80552 Elapsed time : 4 sec
Training // Epoch 167 //  Total cost = 223453.30 Elapsed time : 8 sec
Testing // Epoch 167 //  Total cost = 326767.50  RMSE = 0.80782 Elapsed time : 4 sec
